In [1]:
import numpy as np # for numerical operations
from moviepy.editor import VideoFileClip, concatenate
import os
import librosa
import scipy
from scipy import signal
import numpy as np
import matplotlib.pylab as plt

In [15]:
mypath = "E:\\Graduation Project\\Arabic Data Set\\إنتر ضد إمبولي - المباراة كاملة مباشرةً - الدوري الإيطالي 2021_22.mp4"

# الأهداف

In [2]:
def proj(mypath,Accuracy):
    clip = VideoFileClip(mypath)
    cut = lambda i: clip.audio.subclip(i,i+1).to_soundarray(fps=22000)
    volume = lambda array: np.sqrt(((1.0*array)**2).mean())
    volumes = [volume(cut(i)) for i in range(0,int(clip.duration-1))]
    averaged_volumes = np.array([sum(volumes[i:i+10])/10 for i in range(len(volumes)-10)])
    increases = np.diff(averaged_volumes)[:-1]>=0
    decreases = np.diff(averaged_volumes)[1:]<=0
    peaks_times = (increases * decreases).nonzero()[0]
    peaks_vols = averaged_volumes[peaks_times]
    peaks_times = peaks_times[peaks_vols>np.percentile(peaks_vols,100-Accuracy)]
    final_times=[peaks_times[0]]
    for t in peaks_times:
        if (t - final_times[-1]) < 60:
            if averaged_volumes[t] > averaged_volumes[final_times[-1]]:
                final_times[-1] = t
        else:
            final_times.append(t)
    return final_times

In [5]:
def Goals(mypath,Accuracy=10,ShotSize=10):
    clip = VideoFileClip(mypath)
    final_times = proj(mypath,Accuracy)
    final = concatenate([clip.subclip(max(t-int(ShotSize/2),0),min(t+int(ShotSize/2), clip.duration))
                         for t in final_times])
    path = mypath[mypath.rindex("\\"):]
    final.to_videofile('E:\\Graduation Project\\test2'+path)
    return 'E:\\Graduation Project\\test2'+path

In [10]:
# Goals("E:\\Graduation Project\\Arabic Data Set\\إنتر ضد إمبولي - المباراة كاملة مباشرةً - الدوري الإيطالي 2021_22.mp4",5,15)

# أخطاء

In [30]:
def get_psd_for_all_windows_for_all_seconds(samples,sample_rate):
    results=[]
    for i in range(0,len(samples)//sample_rate):
        start_of_current_window=i*sample_rate
        end_of_current_window=i*sample_rate+640
        result=[]
        for j in range(0,50):
            
            freqs, psd = signal.welch(samples[start_of_current_window:end_of_current_window])
            freqs=freqs*sample_rate
            list_of_whistle_range=[i for i in range(len(freqs)) if ((freqs[i] >= 3500) and (freqs[i]<=4500))]
            #freq_whistle_range=freqs[list_of_whistle_range]
            psd_whistle_range=psd[list_of_whistle_range]
            result.append(sum(psd_whistle_range))
            start_of_current_window+=320
            end_of_current_window+=320
        results.append(result)
    return results      

In [31]:
def is_second_with_whisle(psds_of_second,thre1,thre2):
    for i in range(0,len(psds_of_second)-thre2):
        temp_range=psds_of_second[i:i+thre2]
        if all(temp>=thre1 for temp in temp_range):
            return True
        i+=1
    return False

In [32]:
def get_all_seconds_with_whistle(samples,sample_rate=16000,thre1=0.01,thre2=12):
    accpeted_seconds=[]
    results=get_psd_for_all_windows_for_all_seconds(samples,sample_rate)
    for i in range(len(results)):
        if is_second_with_whisle(results[i],thre1,thre2):
            accpeted_seconds.append(i+1)
    return accpeted_seconds        

In [33]:
def Whistle(mypath,ShotSize=10):
    clip = VideoFileClip(mypath)
    path = mypath[mypath.rindex("\\"):]
    patha = path[:path.rindex(".")]
    clip.audio.write_audiofile('E:\\Graduation Project\\test3'+patha+".wav")
    samples,sr = librosa.load('E:\\Graduation Project\\test3'+patha+".wav",sr=16000)
    TimesFromWhistle = get_all_seconds_with_whistle(samples) 
    acx = []
    for t in range(0,len(TimesFromWhistle)-2,3):
        if (TimesFromWhistle[t+1]-TimesFromWhistle[t]<=(ShotSize/3)) and (TimesFromWhistle[t+2]-TimesFromWhistle[t+1]<=(ShotSize/2)):
            acx.append(TimesFromWhistle[t+1])
        elif TimesFromWhistle[t+1]-TimesFromWhistle[t]<=(ShotSize/3):
            acx.append(TimesFromWhistle[t+1])
            acx.append(TimesFromWhistle[t+2])
        elif TimesFromWhistle[t+2]-TimesFromWhistle[t+1]<=(ShotSize/2):
            acx.append(TimesFromWhistle[t])
            acx.append(TimesFromWhistle[t+1])
        else:
            acx.append(TimesFromWhistle[t])
            acx.append(TimesFromWhistle[t+1])
            acx.append(TimesFromWhistle[t+2])
            
    
    final = concatenate([clip.subclip(max(t-int(ShotSize/3),0),min(t+int(ShotSize/2), clip.duration))
                     for t in acx])
    final.to_videofile('E:\\Graduation Project\\test3'+path)
    return 'E:\\Graduation Project\\test3'+path

In [34]:
Whistle(mypath,ShotSize=10)

                                                                                                                       
chunk:   0%|                                                        | 27/165553 [20:06<2054:56:33, 44.69s/it, now=None]

chunk:   0%|                                                                      | 0/165553 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in E:\Graduation Project\test3\إنتر ضد إمبولي - المباراة كاملة مباشرةً - الدوري الإيطالي 2021_22.wav




chunk:   0%|                                                            | 29/165553 [00:00<16:56, 162.84it/s, now=None]

chunk:   0%|                                                           | 279/165553 [00:00<12:10, 226.30it/s, now=None]

chunk:   0%|▏                                                          | 526/165553 [00:00<08:50, 310.98it/s, now=None]

chunk:   0%|▎                                                          | 779/165553 [00:00<06:30, 421.94it/s, now=None]

chunk:   1%|▎                                                         | 1027/165553 [00:00<04:53, 561.49it/s, now=None]

chunk:   1%|▍                                                         | 1295/165553 [00:00<03:43, 735.93it/s, now=None]

chunk:   1%|▌                                                         | 1555/165553 [00:00<02:55, 936.92it/s, now=None]

chunk:   1%|▋                                                        | 1855/165553 [00:00<02:20, 1162.43it/s, now=None]

chunk:   1%|▋                 

chunk:  10%|█████▋                                                  | 16932/165553 [00:06<01:02, 2384.11it/s, now=None]

chunk:  10%|█████▊                                                  | 17181/165553 [00:07<01:01, 2408.75it/s, now=None]

chunk:  11%|█████▉                                                  | 17442/165553 [00:07<01:00, 2460.70it/s, now=None]

chunk:  11%|█████▉                                                  | 17707/165553 [00:07<00:58, 2511.35it/s, now=None]

chunk:  11%|██████                                                  | 18000/165553 [00:07<00:57, 2555.56it/s, now=None]

chunk:  11%|██████▏                                                 | 18283/165553 [00:07<00:56, 2588.31it/s, now=None]

chunk:  11%|██████▎                                                 | 18546/165553 [00:07<00:56, 2596.74it/s, now=None]

chunk:  11%|██████▎                                                 | 18807/165553 [00:07<00:57, 2572.09it/s, now=None]

chunk:  12%|██████▍             

chunk:  21%|███████████▋                                            | 34368/165553 [00:13<00:46, 2803.01it/s, now=None]

chunk:  21%|███████████▋                                            | 34660/165553 [00:13<00:46, 2830.02it/s, now=None]

chunk:  21%|███████████▊                                            | 35011/165553 [00:13<00:45, 2889.23it/s, now=None]

chunk:  21%|███████████▉                                            | 35311/165553 [00:14<00:45, 2889.33it/s, now=None]

chunk:  22%|████████████                                            | 35612/165553 [00:14<00:45, 2869.24it/s, now=None]

chunk:  22%|████████████▏                                           | 35912/165553 [00:14<00:44, 2892.01it/s, now=None]

chunk:  22%|████████████▏                                           | 36213/165553 [00:14<00:44, 2907.58it/s, now=None]

chunk:  22%|████████████▎                                           | 36513/165553 [00:14<00:44, 2870.60it/s, now=None]

chunk:  22%|████████████▍       

...

t:  89%|███████████████████████████████████████████████████████▏      | 64360/72240 [02:59<00:28, 278.26it/s, now=None]

t:  89%|███████████████████████████████████████████████████████▎      | 64399/72240 [02:59<00:25, 304.26it/s, now=None]

t:  89%|███████████████████████████████████████████████████████▎      | 64439/72240 [02:59<00:23, 327.60it/s, now=None]

t:  89%|███████████████████████████████████████████████████████▎      | 64482/72240 [02:59<00:22, 352.60it/s, now=None]

t:  89%|███████████████████████████████████████████████████████▍      | 64525/72240 [02:59<00:20, 372.49it/s, now=None]

t:  89%|███████████████████████████████████████████████████████▍      | 64576/72240 [03:00<00:27, 276.96it/s, now=None]

t:  89%|███████████████████████████████████████████████████████▍      | 64628/72240 [03:00<00:23, 321.15it/s, now=None]

t:  90%|███████████████████████████████████████████████████████▌      | 64675/72240 [03:00<00:21, 352.54it/s, now=None]

t:  90%|████████████████████████

t:  93%|█████████████████████████████████████████████████████████▉    | 67437/72240 [03:07<00:10, 447.62it/s, now=None]

t:  93%|█████████████████████████████████████████████████████████▉    | 67490/72240 [03:07<00:14, 337.40it/s, now=None]

t:  94%|█████████████████████████████████████████████████████████▉    | 67551/72240 [03:07<00:12, 378.42it/s, now=None]

t:  94%|██████████████████████████████████████████████████████████    | 67604/72240 [03:08<00:11, 413.71it/s, now=None]

t:  94%|██████████████████████████████████████████████████████████    | 67653/72240 [03:08<00:10, 433.83it/s, now=None]

t:  94%|██████████████████████████████████████████████████████████    | 67702/72240 [03:08<00:10, 449.17it/s, now=None]

t:  94%|██████████████████████████████████████████████████████████▏   | 67750/72240 [03:08<00:09, 457.90it/s, now=None]

t:  94%|██████████████████████████████████████████████████████████▏   | 67807/72240 [03:08<00:09, 468.07it/s, now=None]

t:  94%|████████████████████████

t:  98%|████████████████████████████████████████████████████████████▊ | 70887/72240 [03:16<00:03, 385.83it/s, now=None]

t:  98%|████████████████████████████████████████████████████████████▉ | 70941/72240 [03:16<00:03, 407.17it/s, now=None]

t:  98%|████████████████████████████████████████████████████████████▉ | 70996/72240 [03:16<00:02, 441.31it/s, now=None]

t:  98%|████████████████████████████████████████████████████████████▉ | 71051/72240 [03:16<00:03, 327.35it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████ | 71106/72240 [03:16<00:03, 372.49it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████ | 71156/72240 [03:17<00:02, 403.30it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████ | 71215/72240 [03:17<00:02, 430.45it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████▏| 71269/72240 [03:17<00:02, 440.84it/s, now=None]

t:  99%|████████████████████████

Moviepy - Done !
Moviepy - video ready E:\Graduation Project\test3\إنتر ضد إمبولي - المباراة كاملة مباشرةً - الدوري الإيطالي 2021_22.mp4


'E:\\Graduation Project\\test3\\إنتر ضد إمبولي - المباراة كاملة مباشرةً - الدوري الإيطالي 2021_22.mp4'

# أخطاء و أهداف

In [36]:
def GoleWhistle(mypath,Accuracy=10,ShotSize=10):
    clip = VideoFileClip(mypath)
    path = mypath[mypath.rindex("\\"):]
    patha = path[:path.rindex(".")]
    clip.audio.write_audiofile('E:\\Graduation Project\\test4'+patha+".wav")
    samples,sr = librosa.load('E:\\Graduation Project\\test4'+patha+".wav",sr=16000)
    TimesFromWhistle = get_all_seconds_with_whistle(samples)
    TimesFromGolesSum = proj(mypath,Accuracy)    
    alls = set(TimesFromGolesSum+TimesFromWhistle)
    ALL = list(alls)
    ALL.sort()
    acx = []
    for t in range(0,len(ALL)-2,3):
        if (ALL[t+1]-ALL[t]<=(ShotSize/3)) and (ALL[t+2]-ALL[t+1]<=(ShotSize/2)):
            acx.append(ALL[t+1])
        elif ALL[t+1]-ALL[t]<=(ShotSize/3):
            acx.append(ALL[t+1])
            acx.append(ALL[t+2])
        elif ALL[t+2]-ALL[t+1]<=(ShotSize/2):
            acx.append(ALL[t])
            acx.append(ALL[t+1])
        else:
            acx.append(ALL[t])
            acx.append(ALL[t+1])
            acx.append(ALL[t+2])
    li = []
    path = mypath[mypath.rindex("\\"):]
    for t in acx:
        if t in TimesFromGolesSum:
            li.append(clip.subclip(max(t-int(ShotSize/2),0),min(t+int(ShotSize/2), clip.duration)))
        else:
            li.append(clip.subclip(max(t-int(ShotSize/3),0),min(t+int(ShotSize/2), clip.duration)))
    final = concatenate(li)
    final.to_videofile('E:\\Graduation Project\\test4'+path)
    return 'E:\\Graduation Project\\test4'+path

In [37]:
GoleWhistle(mypath,Accuracy=10,ShotSize=10)

                                                                                                                       
chunk:   0%|                                                        | 27/165553 [43:08<4408:06:53, 95.87s/it, now=None]

chunk:   0%|                                                                      | 0/165553 [00:00<?, ?it/s, now=None]

chunk:   0%|                                                          | 165/165553 [00:00<01:50, 1495.81it/s, now=None]

MoviePy - Writing audio in E:\Graduation Project\test4\إنتر ضد إمبولي - المباراة كاملة مباشرةً - الدوري الإيطالي 2021_22.wav




chunk:   0%|▏                                                         | 426/165553 [00:00<01:36, 1713.17it/s, now=None]

chunk:   0%|▏                                                         | 703/165553 [00:00<01:25, 1928.60it/s, now=None]

chunk:   1%|▎                                                        | 1012/165553 [00:00<01:18, 2104.82it/s, now=None]

chunk:   1%|▍                                                        | 1286/165553 [00:00<01:12, 2257.42it/s, now=None]

chunk:   1%|▌                                                        | 1557/165553 [00:00<01:09, 2375.06it/s, now=None]

chunk:   1%|▋                                                        | 1822/165553 [00:00<01:06, 2450.61it/s, now=None]

chunk:   1%|▋                                                        | 2088/165553 [00:00<01:05, 2506.96it/s, now=None]

chunk:   1%|▊                                                        | 2406/165553 [00:00<01:03, 2577.94it/s, now=None]

chunk:   2%|▉                 

chunk:  11%|██████                                                  | 18054/165553 [00:07<01:00, 2420.93it/s, now=None]

chunk:  11%|██████▏                                                 | 18333/165553 [00:07<01:03, 2302.22it/s, now=None]

chunk:  11%|██████▎                                                 | 18629/165553 [00:07<01:01, 2375.46it/s, now=None]

chunk:  11%|██████▍                                                 | 18884/165553 [00:07<01:06, 2220.52it/s, now=None]

chunk:  12%|██████▍                                                 | 19134/165553 [00:07<01:06, 2191.46it/s, now=None]

chunk:  12%|██████▌                                                 | 19375/165553 [00:07<01:05, 2245.23it/s, now=None]

chunk:  12%|██████▋                                                 | 19635/165553 [00:07<01:07, 2162.07it/s, now=None]

chunk:  12%|██████▋                                                 | 19857/165553 [00:07<01:07, 2163.66it/s, now=None]

chunk:  12%|██████▊             

chunk:  22%|████████████▏                                           | 36021/165553 [00:14<00:46, 2760.40it/s, now=None]

chunk:  22%|████████████▎                                           | 36313/165553 [00:14<00:46, 2798.12it/s, now=None]

chunk:  22%|████████████▍                                           | 36644/165553 [00:14<00:45, 2819.01it/s, now=None]

chunk:  22%|████████████▌                                           | 36959/165553 [00:14<00:46, 2783.60it/s, now=None]

chunk:  23%|████████████▌                                           | 37264/165553 [00:14<00:46, 2731.07it/s, now=None]

chunk:  23%|████████████▋                                           | 37565/165553 [00:14<00:47, 2681.85it/s, now=None]

chunk:  23%|████████████▊                                           | 37866/165553 [00:14<00:48, 2651.72it/s, now=None]

chunk:  23%|████████████▉                                           | 38145/165553 [00:14<00:47, 2685.07it/s, now=None]

chunk:  23%|████████████▉       

...

t:  92%|████████████████████████████████████████████████████████▊     | 74863/81720 [03:35<00:19, 351.10it/s, now=None]

t:  92%|████████████████████████████████████████████████████████▊     | 74907/81720 [03:35<00:19, 344.96it/s, now=None]

t:  92%|████████████████████████████████████████████████████████▊     | 74947/81720 [03:36<00:27, 248.70it/s, now=None]

t:  92%|████████████████████████████████████████████████████████▉     | 74996/81720 [03:36<00:23, 289.78it/s, now=None]

t:  92%|████████████████████████████████████████████████████████▉     | 75045/81720 [03:36<00:20, 323.44it/s, now=None]

t:  92%|████████████████████████████████████████████████████████▉     | 75093/81720 [03:36<00:18, 352.63it/s, now=None]

t:  92%|█████████████████████████████████████████████████████████     | 75140/81720 [03:36<00:17, 372.71it/s, now=None]

t:  92%|█████████████████████████████████████████████████████████     | 75182/81720 [03:36<00:26, 249.75it/s, now=None]

t:  92%|████████████████████████

t:  95%|██████████████████████████████████████████████████████████▉   | 77636/81720 [03:44<00:15, 257.39it/s, now=None]

t:  95%|██████████████████████████████████████████████████████████▉   | 77682/81720 [03:44<00:13, 291.32it/s, now=None]

t:  95%|██████████████████████████████████████████████████████████▉   | 77724/81720 [03:44<00:12, 309.02it/s, now=None]

t:  95%|██████████████████████████████████████████████████████████▉   | 77760/81720 [03:44<00:12, 320.80it/s, now=None]

t:  95%|███████████████████████████████████████████████████████████   | 77803/81720 [03:44<00:11, 334.64it/s, now=None]

t:  95%|███████████████████████████████████████████████████████████   | 77840/81720 [03:45<00:15, 246.68it/s, now=None]

t:  95%|███████████████████████████████████████████████████████████   | 77886/81720 [03:45<00:13, 281.87it/s, now=None]

t:  95%|███████████████████████████████████████████████████████████   | 77923/81720 [03:45<00:12, 303.21it/s, now=None]

t:  95%|████████████████████████

t:  98%|████████████████████████████████████████████████████████████▉ | 80298/81720 [03:52<00:05, 249.78it/s, now=None]

t:  98%|████████████████████████████████████████████████████████████▉ | 80338/81720 [03:52<00:05, 276.05it/s, now=None]

t:  98%|████████████████████████████████████████████████████████████▉ | 80371/81720 [03:52<00:04, 289.44it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████ | 80404/81720 [03:53<00:04, 297.49it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████ | 80439/81720 [03:53<00:04, 310.91it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████ | 80472/81720 [03:53<00:03, 312.16it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████ | 80505/81720 [03:53<00:03, 307.80it/s, now=None]

t:  99%|█████████████████████████████████████████████████████████████ | 80537/81720 [03:53<00:05, 220.29it/s, now=None]

t:  99%|████████████████████████

Moviepy - Done !
Moviepy - video ready E:\Graduation Project\test4\إنتر ضد إمبولي - المباراة كاملة مباشرةً - الدوري الإيطالي 2021_22.mp4


'E:\\Graduation Project\\test4\\إنتر ضد إمبولي - المباراة كاملة مباشرةً - الدوري الإيطالي 2021_22.mp4'

# أخطاء معينة